# Building a new universal model for prokaryotes from the BIGG universe

Two models: gram-negative and gram-positive. Gram-negative contains: extracellular, periplasm and cytoplasm. Gram-positive only extracellular and cytoplasm.

1) Gran-negative, take all reactions that are not in those compartments and move them to cytoplasm. Remove shuttle/transport reactions between compartments that are not in gram neg

2) Remove duplicated reactions/metabolites in the model

In [2]:
import cobra
from cobra import Reaction, Model


def remove_shuttle_reactions(model: Model, allowed_compartments: set = {"c", "e", "p"}) -> Model:
    """
    Remove shuttle reactions between unwanted compartments.

    Args:
        model (Model): _description_
        allowed_compartments (set, optional): _description_. Defaults to {"c", "e", "p"}.

    Returns:
        Model: _description_
    """
    shuttle_rxns_in_unwanted_compartments = [
        rxn for rxn in model.reactions
        if (
            (len(rxn.compartments) > 1) and 
            (not rxn.compartments.issubset(allowed_compartments))
        )
        ]
    model.remove_reactions(shuttle_rxns_in_unwanted_compartments, remove_orphans=True)
    return model

def move_reactions_to_cytoplasm(model: Model, allowed_compartments: set = {"c", "e", "p"}) -> Model:
    """
    Update the metabolites of a reaction to include a new set of metabolites
    Args:
        reaction (Reaction): _description_
        allowed_compartments (set, optional): _description_. Defaults to {"c", "e", "p"}.

    Returns:
        Reaction: _description_
    """
    reactions_to_add = []
    reactions_to_remove = []
    for reaction in model.reactions:
        if not reaction.compartments.issubset(allowed_compartments):

            new_metabolites = {}
            for metabolite, stoich in reaction.metabolites.items():
                new_met_id = metabolite.id[:-1] + 'c'
                new_metabolite = model.metabolites.get_by_id(new_met_id) if new_met_id in model.metabolites else metabolite.copy()
                new_metabolite.compartment = 'c'
                new_metabolite.id = new_met_id
                if new_met_id not in model.metabolites:
                    model.add_metabolites([new_metabolite])
                    model.remove_metabolites([metabolite])
                new_metabolites[new_metabolite] = stoich

            new_reaction = Reaction(
                id=reaction.id,
                name=reaction.name,
                lower_bound=reaction.lower_bound,
                upper_bound=reaction.upper_bound,
                subsystem=reaction.subsystem,
                )
            new_reaction.gene_reaction_rule = reaction.gene_reaction_rule
            new_reaction.add_metabolites(new_metabolites)
            reactions_to_add.append(new_reaction)
            reactions_to_remove.append(reaction)

    model.remove_reactions(reactions_to_remove, remove_orphans=True)
    model.add_reactions(reactions_to_add)
    return model

def test_remove_shuttle_reactions(model: Model, allowed_compartments: set = {}) -> bool:
    """"""
    return True if not [
        rxn for rxn in model.reactions if not rxn.compartments.issubset(allowed_compartments)
        ] else False

In [3]:
unineg = cobra.io.read_sbml_model('carveme_universes/BIGG_universal_model/universal_model_cobrapy.xml')
unineg

Set parameter Username


Academic license - for non-commercial use only - expires 2023-11-05


Name,bigg_universal
Memory address,7f33303012d0
Number of metabolites,15638
Number of reactions,28301
Number of genes,0
Number of groups,0
Objective expression,1.0*BIOMASS_reaction - 1.0*BIOMASS_reaction_reverse_5a818
Compartments,"cytoplasm, extracellular, periplasm, mitochondrion, peroxisome, unknown, nucleus, vacuole, golgi, thylakoid, lysosome, chloroplast, eyespot, flagellum, mitochondrial intermembrane space, unknown, unknown, unknown, unknown, mitochondrial membrane, cell wall, unknown"


## Remove unwanted shuttle reactions

In [3]:
gramneg_compartments = {"e", "c", "p"}
unineg = remove_shuttle_reactions(unineg, allowed_compartments=gramneg_compartments)
unineg

Name,bigg_universal
Memory address,7fb3c39e6dd0
Number of metabolites,15467
Number of reactions,25787
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,"cytoplasm, extracellular, periplasm, mitochondrion, peroxisome, unknown, nucleus, vacuole, golgi, thylakoid, lysosome, chloroplast, eyespot, flagellum, unknown, unknown, cell wall, unknown"


## Move reactions to cytoplasm

In [4]:
unineg = move_reactions_to_cytoplasm(unineg)
unineg

Name,bigg_universal
Memory address,7fb3c39e6dd0
Number of metabolites,11970
Number of reactions,25787
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,"cytoplasm, extracellular, periplasm"


In [5]:
# Test if any reaction remains in illegal comparments
print("passed" if test_remove_shuttle_reactions(unineg, gramneg_compartments) else "failed")

passed


## Add biomass reaction to universal gram negative model

Take reaction from universal CarveME gram negative model

In [6]:
carveme_gramneg = cobra.io.read_sbml_model("carveme_universes/universe_gramneg.xml")

In [7]:
growth = carveme_gramneg.reactions.get_by_id("Growth")
unineg.add_reactions([growth])
unineg.objective = "Growth"
unineg

Name,bigg_universal
Memory address,7fb3c39e6dd0
Number of metabolites,11970
Number of reactions,25788
Number of genes,0
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytoplasm, extracellular, periplasm"


In [ ]:
# Open exchanges
unineg.open_exchanges()

In [12]:
# Test growth
unineg.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


## Run memote

In [8]:
cobra.io.write_sbml_model(unineg, "carveme_universes/universal_gramnegative.xml")

In [11]:
# %%bash

# memote run \
#     --filename carveme_universes/universal_gramnegative_report.json \
#     --ignore-git \
#     carveme_universes/universal_gramnegative.xml

Set parameter Username


Set parameter Username


Academic license - for non-commercial use only - expires 2023-11-05


Academic license - for non-commercial use only - expires 2023-11-05


## Remove duplicated reactions

__NOTE__:

Perhaps makes more sense to remove duplicated reactions after carving since reactions in the universal model have no associated genes. Thus reactions catalyzed by different enzymes are targeted as duplicated by memote.

In [ ]:
import json
from pathlib import Path
from cobra import Model


# Implemented in memote
def get_duplicated_reactions(memote_report: Path) -> list[list]:
    """Retrieve duplicated reactions from memote report

    Args:
        memote_report (Path): path to report (JSON file)

    Returns:
        list: list of lists of duplicated reaction pair IDs
    """
    with open(memote_report, "r") as f:
        report = json.load(f)
    tests = report["tests"]
    return tests["test_find_duplicated_reactions"]["data"]

def remove_duplicated_reactions(
        model: Model,
        duplicated_reactions: list[list],
        inplace: bool = True) -> Model:
    """Remove duplicated reactions from model

    Args:
        model (Model): cobra model object
        duplicated_reactions (list[list]):list lists of duplicated pairs
        inplace (bool, optional): removes reactions in the input model or return a new one.
            Defaults to True.

    Returns:
        Model: model object without duplicated reactions
    """
    if not inplace:
        result_model = model.copy()
    else:
        result_model = model
    reactions_to_remove = [rxn_pair[0] for rxn_pair in duplicated_reactions]
    result_model.remove_reactions(reactions_to_remove, remove_orphans=True)
    return result_model


# unineg_report = Path("carveme_universes/universal_gramnegative_report.json")
# unineg_duplicated_rxns = get_duplicated_reactions(unineg_report)
# unineg_duplicated_rxns


In [ ]:
# unineg = remove_duplicated_reactions(unineg, unineg_duplicated_rxns)

In [1]:
# import numpy as np


# def find_duplicated_reactions(model: Model) -> list[tuple]:
#     """Find duplicated reactions in a model."""
#     duplicated_reactions = []
#     for rxn_i in model.reactions:
#         dup_i = {rxn_j for rxn_j in model.reactions if rxn_i.metabolites == rxn_j.metabolites}.add(rxn_i)
#     duplicated_reactions.append(dup_i)
#     return list(np.unique(duplicated_reactions))


# unineg_duplicated_rxns = find_duplicated_reactions(unineg)
# unineg_duplicated_rxns

## Write universal model

In [ ]:
cobra.io.write_sbml_model(unineg, "carveme_universes/universal_gramnegative.xml")